**If you are not familiar with jupyter notebook read this intoduction https://realpython.com/jupyter-notebook-introduction/**

**Read the introduction section of the paper**

# Settings

Importing python packages

In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rcParams
import plotly.graph_objects as go
from pingouin import mediation_analysis
from statannot import add_stat_annotation
from scipy.stats import pearsonr, mannwhitneyu, wilcoxon

In [ ]:
# allows the output of the code to be displayed within the jupyter cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

Figure parameters

In [ ]:
rcParams['font.size'] = 12
rcParams['figure.figsize'] = (9, 6)
rcParams['savefig.dpi'] = 300
rcParams['savefig.format'] = 'png'

Study parameters

In [ ]:
colors = {
    'MED diet': 'cornflowerblue', 
    'PPT diet': 'orange',
    'Gut': 'lightpink',
    'Oral': 'mediumseagreen'}

In [ ]:
diet_order = ['PPT diet', 'MED diet']
time_point_order = ['Pre-intervention', 'Post-intervention']
env_order = ['Oral', 'Gut']
mediation_order = ['diet-species-glucose', 'diet-species-serum', 
                   'diet-pathways-glucose', 'diet-pathways-serum', 
                   'diet-strains-glucose', 'diet-strains-serum']

In [ ]:
# the order they are introduced in the paper
data_types = ['diet', 'gut species', 'gut pathways', 'oral species', 'oral pathways', 'metabolites', 'cytokines']

In [ ]:
alpha = 0.05  # of statistical tests

In [ ]:
# function to calculate the difference in feature values between the post- and pre- intervention samples
def get_delta(df):
   
    pre = df.xs('Pre-intervention', level='Time Point')
    post = df.xs('Post-intervention', level='Time Point')

    return post.subtract(pre)

# Data

**If you are not familiar with numpy go over this tutorial https://numpy.org/doc/stable/user/absolute_beginners.html**

**If you are not familiar with pandas go over these tutorials https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html**

**Read the results section titled "Dietary interventions in pre-diabetes"**

Load the excel file called basic.xlsx using the pandas package

Set the Diet, Participant ID and Time Point columns as index

Here is an example of how to do it

In [ ]:
file_name = 'basic.xlsx'
basic = pd.read_excel(file_name)
basic.head()

In [ ]:
index = ['Diet', 'Participant ID', 'Time Point']
basic = basic.set_index(index)
basic.head()

Explore the data frame

In [ ]:
basic

Does it have the expected amount of participants mentioned in the paper? in each diet group? and in each time point?

Here is an example of how to do it

In [ ]:
for i in index:
    index_values = basic.index.get_level_values(i)
    index_values.value_counts()

Does it have missing values? which feature and which person have the most missing values?

Here is an example of how to do it

In [ ]:
basic.isna()

In [ ]:
basic.isna().sum()

In [ ]:
basic.isna().sum().sort_values(ascending=False)

In [ ]:
basic.isna().sum(axis=1).sort_values(ascending=False)

Calculate the in person change of blood test (bt) glygcated hemoglobin (hba1c)

Compare the means of the two diet groups

Here is an example of how to do it

In [ ]:
get_delta(basic)

In [ ]:
get_delta(basic)['bt__hba1c']

In [ ]:
get_delta(basic)['bt__hba1c'].mean()

In [ ]:
get_delta(basic)['bt__hba1c'].groupby('Diet').mean()

## Table 1

Re-create Table 1 of the paper based on "basic.xlsx"

Here is an example of how to do it

In [ ]:
basic.xs('Pre-intervention', level='Time Point')

In [ ]:
basic.xs('Pre-intervention', level='Time Point').groupby('Diet').mean()

In [ ]:
basic.xs('Pre-intervention', level='Time Point').groupby('Diet').mean().T

In [ ]:
basic.xs('Pre-intervention', level='Time Point').groupby('Diet').mean().T[diet_order]

In [ ]:
basic.xs('Pre-intervention', level='Time Point').groupby('Diet').std().T[diet_order]

**Read the method section titled "Data Processing"**\
It referes to the data in Supplementary file 1

The code below loads all the data from the supplementary files, it can take a few minutes

In [ ]:
data = {}

s1 = 'Supplementary file 1 - Source data.xlsx'
index = ['Diet', 'Participant ID', 'Time Point']
for data_type in data_types:
    header = 12 if data_type == 'metabolites' else 0
    data[data_type] = pd.read_excel(s1, sheet_name=data_type, header=header).set_index(index)

s2 = 'Supplementary file 2 - Statistical tests.xlsx'
index = ['Diet', 'Participant ID', 'Time Point']
for data_type in data_types:
    for diet in diet_order:
        data[f'{data_type} {diet}'] = pd.read_excel(s2, sheet_name=f'{data_type} {diet}').set_index('feature')

s3 = 'Supplementary file 3 - Mediation analyses.xlsx'
index = ['path', 'x', 'm', 'y']
for med in mediation_order:
    suffix = env_order if 'strains' in med else diet_order
    for suf in suffix:
        data[f'{med} {suf}'] = pd.read_excel(s3, sheet_name=f'{med} {suf.split(" ")[0]}', index_col=[0, 1, 2, 3])
        
s4 = 'Supplementary file 4 - Metabolites prediction by the microbiome.xlsx'
index = ['Diet', 'Participant ID', 'Time Point']
data_type = 'metabolites prediction'
data[data_type] = pd.read_excel(s4, sheet_name=data_type, header=[0, 1]).T.reset_index(0).T.set_index(index)

s5 = 'Supplementary file 5 - Oral and gut microbial strains.xlsx'
index = ['Species', 'Participant ID', 'Diet']
for env in env_order:
    data_type = f'{env.lower()} strains'
    data[data_type] = pd.read_excel(s5, sheet_name=data_type).set_index(index)

The data is saved in a dictionary, with each value being a data frame

In [ ]:
for key in data.keys():
    print(key, data[key].shape)

For example

In [ ]:
data['diet']

# Visualization

**if you are not familiar with matplotlib pyplot go over this tutorial https://matplotlib.org/stable/tutorials/introductory/pyplot.html**

**if you are not familiar with seaborn go over this tutorial https://seaborn.pydata.org/tutorial/introduction.html**

## Figure 2

Re-create Figure2 of the paper based on data['diet']
- Play with the hue_order, palette and alpha seaborn parameters
- Move the legend to each of the corners
- Set the axes limit of both sub-plots to be the same
- Read about the plt.text transform parameter

Here is an example of how to do it

In [ ]:
df = data['diet'].reset_index()

In [ ]:
for time_point in time_point_order:
    
    plt.figure()
    ax = sns.scatterplot(x='% Carbohydrates', y='% Lipids', hue='Diet', hue_order=diet_order, data=df[df['Time Point'] == time_point], palette=colors, alpha=0.5, s=100)

    plt.legend(title=False, loc='upper right', frameon=True)

    plt.title('Intervention' if time_point == 'Post-intervention' else time_point)
    plt.xlabel('% Carbohydrates in diet')
    plt.ylabel('% Lipids in diet')
    plt.xlim([9, 55])
    plt.ylim([15, 75])
    
    plt.text(x=0, y=1.03, s='a' if time_point == 'Pre-intervention' else 'b', transform=ax.transAxes, size=20, weight='bold')   

    plt.savefig(f'Figure 2{"a" if time_point == "Pre-intervention" else "b"}')

# Statistical tests

**Refresh your memory on statistical tests https://doi.org/10.4103/0974-7788.72494**

Which test would you chose for between diet group comparison? why?

Was the basline age, gender and BMI the same between the groups?

Here is an example of how to do it

In [ ]:
for col in ['Age', 'Gender', 'BMI']:
    col
    local_df = basic.xs('Pre-intervention', level='Time Point')[col].dropna()
    mannwhitneyu(local_df.loc[diet_order[0]], local_df.loc[diet_order[1]])

Which test would you chose for within diet group comparison? why?

Within each of the diet groups check if there was a signfiicant difference in the three glycemic measurments

Here is an example of how to do it

In [ ]:
for diet in diet_order:
    diet
    
    for col in ['Time_above_140', 'bt__hba1c', 'OGTT']:
        col
        
        local_df = basic.loc[diet, col]
        nan_participants = local_df[local_df.isna()].index.get_level_values('Participant ID')
        local_df = local_df[~local_df.index.get_level_values('Participant ID').isin(nan_participants)]
        
        mannwhitneyu(local_df.xs(time_point_order[0], level='Time Point'), 
                     local_df.xs(time_point_order[1], level='Time Point'))
        
    print()

## Figure 3

**Read the results section titled "The PPT diet had bigger effect on the microbiome and metabolites than the MED diet"\
(Microbiome, Metabolites and Cytokines)**\
All the statistical tests results described in this section are in Supplementary file 2

**Read the methods sections titled "Feature analysis" and "Statistics"**

This is the gut microbial species relative abundance data

In [ ]:
data['gut species']

The column names represent the species by their phylgentics

In [ ]:
data['gut species'].columns

First at a taxonomical level

In [ ]:
data['gut species'].columns.str.split('|').str[:7]

Looking at one species makes it more clear

In [ ]:
data['gut species'].columns.str.split('|').str[:7][0]

Second from an ID prespective

In [ ]:
data['gut species'].columns.str.split('|').str[7:]

In [ ]:
data['gut species'].columns.str.split('|').str[7:][0]

These are the significantly changed gut species in the PPT arm

In [ ]:
data['gut species PPT diet'][data['gut species PPT diet']['p_bonferroni'] < alpha]

These are the significantly changed gut species in the MED arm

In [ ]:
data['gut species MED diet'][data['gut species MED diet']['p_bonferroni'] < alpha]

Visualize the significant results
- it does not have to be in the same type of figure as in the paper
- but it should include the same aspects of information

For example:
- the species ID (e.g. sSGB__714), name (e.g. Methanobrevibacter smithii) and family (e.g. Methanobacteriaceae)
- the direction of change in each group

Here is an example of how to do it

In [ ]:
def get_data(data_type):
    
    for i, diet in enumerate(diet_order):
        key = f'{data_type} {diet}'
        col = 'p_FDR' if data_type == 'cytokines' else 'p_bonferroni'
        new = get_delta(data[data_type].loc[diet, data[key].index[data[key][col] < alpha]]).mean().to_frame(diet)
        df = new.copy() if i == 0 else df.join(new, how='outer').fillna(0)
        
    return df

In [ ]:
metabolite2super_pathway = pd.read_excel(s1, sheet_name='metabolites')
metabolite2super_pathway = metabolite2super_pathway.set_index('PATHWAY_SORTORDER').loc[['Diet', 'SUPER_PATHWAY']].T.set_index('Diet').fillna('unknown').iloc[2:].replace('Partially Characterized Molecules', 'Partially Characterized').to_dict()['SUPER_PATHWAY']

In [ ]:
for data_type in data_types:
    
    df = get_data(data_type)
        
    if df.shape[0] < 10:
        continue
              
    radius = np.log10(df.shape[0])
    step = radius/8
    
    cmap = plt.get_cmap('bwr')
    norm = mpl.colors.Normalize(vmin=-df.abs().max().max() if data_type != 'diet' else -10, vmax=df.abs().max().max() if data_type != 'diet' else 10)
    
    if data_type == 'diet':
        df['Level'] = 'Food category'
        df.loc[df.index.str.contains('%'), 'Level'] = 'Macro-nutrient'
        df.loc[df.index.str.contains(']'), 'Level'] = 'Micro-nutrient'
        # sort
        df = df.loc[[idx for idx in ['% Carbohydrates', '% Lipids', '% Proteins', '% Daily caloric target', '% Fibers', 
                                     '% Mono un-saturated fatty acids', '% Poly un-saturated fatty acids', '% Saturated fatty acids']
                         if idx in df.index] + 
                     sorted([idx for idx in df.index if '[' in idx]) + 
                     sorted([idx for idx in df.index if '%' not in idx and '[' not in idx])]
    elif 'species' in data_type:
        df['Family'] = df.index.str.split('|').str[-6].str.replace('f__', '').str.replace('_unclassified', '').str.replace('unknown', 'Unclassified')
        df.index = df.index.str.split('|').str[-4].str.replace('s__', '').str.replace('_', ' ').str.replace('unknown', 'Unclassified') + ' (' + df.index.str.split('|').str[-1].str.replace('sSGB__', 'SGB_') + ')'
        df.index = ['Unclassified ' + index.split(' ')[-1] if 'sp' in index or 'CAG' in index else index for index in df.index]
        df = df.sort_values('Family')
    elif data_type == 'metabolites':
        df['Super Pathway'] = df.index.map(metabolite2super_pathway).str.replace('unknown', 'Uncharacterized')
        df = df.sort_values('Super Pathway')
    elif 'pathways' in data_type:
        super_class = {  # curated manually from MetCyc
            'ARGSYNBSUB-PWY: L-arginine biosynthesis II (acetyl cycle)' : 'Amino Acid Biosynthesis',
            'ARGSYN-PWY: L-arginine biosynthesis I (via L-ornithine)': 'Amino Acid Biosynthesis',
            'GLUTORN-PWY: L-ornithine biosynthesis I': 'Amino Acid Biosynthesis',
            'PWY-6292: superpathway of L-cysteine biosynthesis (mammalian)': 'Amino Acid Biosynthesis',
            'PWY-6507: 4-deoxy-L-threo-hex-4-enopyranuronate degradation': 'Sugar Degradation',
            'GALACTUROCAT-PWY: D-galacturonate degradation I': 'Sugar Degradation',
            'PWY-7356: thiamine diphosphate salvage IV (yeast)': 'Cofactor&Vitamin Biosynthesis',#'Cofactor, Carrier, and Vitamin Biosynthesis',
            'GLUCUROCAT-PWY: superpathway of &beta;-D-glucuronosides degradation': 'Sugar Degradation',
            'PWY-7242: D-fructuronate degradation': 'Sugar Degradation',
            'PWY-7456: &beta;-(1,4)-mannan degradation': 'Polysaccharide Degradation',
            'PWY66-399: gluconeogenesis III': 'Sugar Biosynthesis',
            'PWY-7383: anaerobic energy metabolism (invertebrates, cytosol)': 'zOther',#'Fermentation',
            'PWY490-3: nitrate reduction VI (assimilatory)': 'zOther',#'Inorganic Nutrient Metabolism',
            'PWY-6305: superpathway of putrescine biosynthesis': 'zOther',#'Amide, Amidine, Amine, and Polyamine Biosynthesis',
            'P124-PWY: Bifidobacterium shunt': 'Sugar Degradation',
            'PWY-5941: glycogen degradation II': 'Polysaccharide Degradation',
            'PWY-7238: sucrose biosynthesis II': 'Sugar Biosynthesis',
            'PWY0-1296: purine ribonucleosides degradation': 'zOther',#'Nucleoside and Nucleotide Degradation',
            'PWY-6147: 6-hydroxymethyl-dihydropterin diphosphate biosynthesis I': 'zOther',#'Other Biosynthesis',
            'PWY-6549: L-glutamine biosynthesis III': 'zOther',#'Transport',
            'PWY-6703: preQ0 biosynthesis': 'zOther',#'Secondary Metabolite Biosynthesis',
            'PWY-6823: molybdopterin biosynthesis': 'Cofactor&Vitamin Biosynthesis',#'Cofactor, Carrier, and Vitamin Biosynthesis',
            'PWY-241: C4 photosynthetic carbon assimilation cycle, NADP-ME type': 'Carbon Fixation',#'Generation of Precursor Metabolites and Energy',
            'PWY-7115: C4 photosynthetic carbon assimilation cycle, NAD-ME type': 'Carbon Fixation',#'Generation of Precursor Metabolites and Energy',
            'PWY-7117: C4 photosynthetic carbon assimilation cycle, PEPCK type': 'Carbon Fixation'}#'Generation of Precursor Metabolites and Energy'}
        df['Super Class'] = df.index.map(super_class)
        df = df.sort_values('Super Class').replace('zOther', 'Other')
        
    df.loc[' '] = ''
    df.loc['Diet'] = df.columns
    df.loc['   '] = ''

    fig, ax = plt.subplots()

    for i, diet in enumerate(df.columns):
        if diet in diet_order:
            c = cmap(norm(df[diet_order[i]][:-3].astype(float))).tolist() + ['white', colors[diet], 'white']
        else:
            cmap = mpl.cm.Pastel1
            norm = mpl.colors.Normalize(vmin=0, vmax=8)
            c = {cat: cmap(norm(i)) for i, cat in enumerate(df[diet][:-3].unique())}
            c = [c[v] for v in df[diet][:-3]] + ['white', 'white', 'white']
        
        pie, labels = ax.pie([1 for i in range(df.shape[0])], colors=c,
                             radius=radius-i*step, startangle=0,
                             labels=df[diet_order[i]].index if i == 0 else None, rotatelabels=True, labeldistance=1,
                             wedgeprops={'width':step, 'edgecolor':'w'})

        _ = plt.setp(pie, width=step, edgecolor='white')
        
    if diet not in diet_order:
        index = df[diet][:-3].reset_index().index[~df[diet][:-3].duplicated()]
        legend = ax.legend([pie[i] for i in index], [df[diet][i] for i in index], 
                           title=diet, loc='center', bbox_to_anchor=(0., 0, 1, 1.), frameon=False)
    
    extra = {'diet':        {'x': -0.40, 'y': 1.55, 'l': 'a', 'title': '     Diet'},
             'gut species': {'x': -0.78, 'y': 1.45, 'l': 'b', 'title': '     Gut species'},
             'gut pathways': {'x': -1.20, 'y': 2.20, 'l': 'c', 'title': '     Gut pathways'},
             'metabolites': {'x': -1.20, 'y': 2.20, 'l': 'd', 'title': '     Metabolites'}}
    
    _ = plt.text(x=extra[data_type]['x'], y=extra[data_type]['y'], s=extra[data_type]['l'], transform=ax.transAxes, size=20, weight='bold')   
    _ = plt.text(x=extra[data_type]['x'], y=extra[data_type]['y'], s=extra[data_type]['title'], transform=ax.transAxes)   
    
    plt.savefig(f'Figure 3{extra[data_type]["l"]}', bbox_inches='tight')

# Mediation analysis

**If you are not familiar with the concept of mediation analysis read this https://data.library.virginia.edu/introduction-to-mediation-analysis/**

**Read the results section titled "The microbiome mediates the diet's effect"**

**Read the methods sections titled "Mediation analysis"**

Implement mediation analysis for this triad in the PPT group:\
X - % Proteins in diet\
M - Gut Faecalibacterium prausnitzii (SGB_15342)\
Y - Time above 140

Use the change in X, M and Y

Regress Y on X, is the effect of X significant?

Regress M on X, is the effect of X significant?

Regress Y on X and M, is the effect of M significant?

Use the pingouin.mediation_analysis package to do the same process, did you get the same results?\
https://pingouin-stats.org/build/html/generated/pingouin.mediation_analysis.html#pingouin.mediation_analysis

Adjust for baseline age, sex and BMI

All the mediation analyses results described in the paper are in Supplementary file 3\
Did you get the same results as in the paper?

In [ ]:
data['diet-species-glucose PPT diet'].xs('% Proteins', level='x').xs('Faecalibacterium prausnitzii (SGB_15342)', level='m').xs('Time_above_140', level='y')

## Figure 4

Visualize the significant results
- it does not have to be in the same type of figure as in the paper
- but it should include the same aspects of information

Here is an example of how to do it

In [ ]:
letters = 'abcd'
l = 0

for med in mediation_order[:2]:

    suffix = env_order[::-1] if 'strains' in med else diet_order
    for suf in suffix:
        
        df = data[f'{med} {suf}'].copy()
        df = df.groupby(['x', 'm', 'y']).filter(lambda g: (g.loc[['M ~ X', 'Indirect'], 'p'] < alpha).all())
        if df.empty:
            continue
        df = df.loc['Indirect'].reset_index(['x', 'm', 'y'])
        df.shape

        label2int = {label: i for i, label in enumerate(list(df['x'].unique()) + list(df['m'].unique()) + list(df['y'].unique()))}

        link = dict(source=pd.concat([df['x'], df['m']]).replace(label2int).values, 
                    
                    target=pd.concat([df['m'], df['y']]).replace(label2int).values,

                    value=[1]*df.shape[0]*2,

                    color=[f'rgba{mpl.cm.tab10(label2int[label]%10)}' for label in df['x']] + \
                          [f'rgba{mpl.cm.tab10(label2int[label]%10)}' for label in df['x']])

        node = dict(pad=35, thickness=20, 
                    label=list(label2int.keys()), 
                    color=[f'rgba{mpl.cm.tab10(label2int[label]%10)}' for label in df['x'].unique()]+['darkgrey']*(len(df['m'].unique())+len(df['y'].unique())))
               
        fig = go.Figure(go.Sankey(link=link, node=node))
        med_nice = med.replace('diet-', ('Diet - '+suf+' ') if 'strains' in med else (suf+' - Microbial ')).replace('-glucose', ' - Glycemic measurements').replace('-serum', ' - Metabolites and Cytokines')
        fig.update_layout(title=dict(text=f'{letters[l]}   {med_nice}', font=dict(color='Black')))
        figure = '4' if 'species' in med else ('S2' if 'pathways' in med else 'S3')
        try:
            fig.write_image(f'Figure {figure}{letters[l]}.png', width=1500)
        except:
            f'FAILED SAVING Figure {figure}{letters[l]}.png'
            
        l = 0 if (l >= 2) & (suf != suffix[0]) else l + 1

# Model evalution

**Read the results section titled "The change in microbiome composition is associated with the change in metabolites"**

**Read the methods sections titled "Model analysis"**

**Read about the distingtion between R2 and r^2 https://towardsdatascience.com/r%C2%B2-or-r%C2%B2-when-to-use-what-4968eee68ed3**

The model used was created as part of this paper: https://doi.org/10.1038/s41586-020-2896-2

The metabolites prediction by the microbiome described in this section are in Supplementary file 4\
Notice that the first row is the R2

In [ ]:
data['metabolites prediction']

Split the data to the well predicted metabolites (R2<0.05 in the training set) and the poorly predicted metabolites (R2>0.05)

Compute the mean predicted change in each metabolite

Compute the mean actual change in each metabolite

In [ ]:
data['metabolites']

Separetly for the well and for the poorly predicted metabolites, compute the expalined variance

## Figure 5

Plot the results

Here is an example of how to do it

In [ ]:
def get_data(r):

    preds = get_delta(data['metabolites prediction'].iloc[1:].astype(float))
    actuals = get_delta(data['metabolites']).loc[preds.index, preds.columns]

    df = actuals.groupby('Diet').mean().melt(ignore_index=False).set_index('variable', append=True).rename(columns={'value': 'observed change'}).join(
         preds.groupby('Diet').mean().melt(ignore_index=False).set_index('variable', append=True).rename(columns={'value': 'predicted change'}))

    df = df.join(data['metabolites prediction'].iloc[0].T.to_frame('R2').astype(float).dropna(), on='variable')
    df = df.loc[df['R2'] < -r] if r < 0 else df.loc[df['R2'] > r]
    df = df.reset_index()

    return df

In [ ]:
for r2 in [-0.05, 0.05]:
    
    df = get_data(r2)
    
    plt.figure()
    size = 'R2' if r2 > 0 else None
    ax = sns.scatterplot(x='observed change', y='predicted change', hue='Diet', hue_order=diet_order, data=df, s=100, size=size, sizes=(50 ,300), palette=colors, alpha=0.5)

    pearsonr(df['observed change'], df['predicted change'])

    l = plt.legend(title=False, loc='lower right', frameon=True)
    for handle in l.legendHandles:
        diet = handle.get_label()
        if diet in diet_order:
            r, p = pearsonr(df.loc[df['Diet'] == diet, 'observed change'], df.loc[df['Diet'] == diet, 'predicted change'])
            r, p
            handle.set_label(f'{diet}\nr={r:.2f}, ' + (f'p<1e{math.floor(math.log10(p))+1}' if p < 0.01 else f'p={p:.2f}'))
    handles = l.legendHandles[1:] if len(l.legendHandles) > 5 else l.legendHandles
    plt.legend(handles=handles, title=False, loc='upper left', frameon=True)
        
    plt.xlabel('Mean observed change')
    plt.ylabel('Mean predicted change')

    df_abs = df.copy()
    df_abs[['observed change', 'predicted change']] = df_abs[['observed change', 'predicted change']].abs()
    index = df_abs.sort_values('observed change').tail(3).index.union(df_abs.sort_values('predicted change').tail(3).index)
    if r2 > 0:
        index = index.union(df.sort_values('R2').tail(3).index)
        index = list(index) + [117]
    for i in index:
        s, x, y = df.loc[i, ['variable', 'observed change', 'predicted change']]
        if i == 856:
            va = 'top'
        elif i == 857:
            va = 'bottom'
        else:
            va = 'center'
        _ = plt.text(x=x, y=y, s=s, va=va)
    
    plt.text(x=0, y=1.03, s='a' if r2 > 0 else 'b', transform=ax.transAxes, size=20, weight='bold')   

    plt.title(f'{"Poorly" if r2 < 0 else "Well"} predicted metabolites (R2{"<" if r2 < 0 else ">"}{abs(r2)})')
    plt.savefig(f'Figure 5{"a" if r2 > 0 else "b"}')

# Sample size

**Read the results section titled "The oral microbiome is genetically more dynamic than the gut microbiome"**

**Read the methods sections titled "Strain analysis"**

In [ ]:
data['gut strains']

Aggregate the results to the species level - i.e. how many people had a species? and how many of those who had it had a "strain replacement"?

Which species had the most strain replacements?

Look only at species that exist in at least 50 people, does your answer change?

Here is an example of how to do it

In [ ]:
local_df = data['gut strains'].copy()
local_df = local_df.groupby('Species')['Strain replacement'].apply(lambda g: (g.shape[0], g.mean()*100)).to_frame()
local_df['n'], local_df['%'] = local_df.iloc[:, 0].str

In [ ]:
local_df.sort_values('%', ascending=False)

In [ ]:
local_df[local_df['n'] >= 50].sort_values('%', ascending=False)

**Read about the law of small numbers https://psychology.iresearchnet.com/social-psychology/decision-making/law-of-small-numbers/#:~:text=The%20law%20of%20small%20numbers,at%20all%2C%20but%20a%20fallacy.**

## Figure 6

**Read about the statannot.add_stat_annotation python package, it can save you a lot of time https://github.com/webermarcolivier/statannot/blob/master/example/example.ipynb**

Re-create Figure6 d of the paper

Here is an example of how to do it

In [ ]:
hue = 'env'
hue_order = env_order

In [ ]:
def get_data(b):
    
    df = pd.concat([data['gut strains'].assign(env='Gut'), data['oral strains'].assign(env='Oral')])
    df['Strain replacement'] = df['Strain replacement'].map({True: 1, False: 0})
    df = df.groupby([b, hue])['Strain replacement'].apply(lambda g: (g.shape[0], g.mean()*100)).to_frame()
    df['n'], df['%'] = df.iloc[:, 0].str
    df['n_bin'] = pd.cut(df['n'], bins=np.arange(0, df['n'].max(), 18 if b == 'Participant ID' else 30))
    df = df.reset_index()
    
    return df

In [ ]:
for by in ['Participant ID', 'Species']:

    df = get_data(by)

    plt.figure()
    ax = sns.histplot(df, x='%', hue=hue, hue_order=hue_order, palette=colors, kde=False, element='step', binwidth=10, alpha=0.3, common_norm=False, stat='percent')

    l = ax.legend(labels=hue_order)
    ax.legend(labels=hue_order, handles=l.legendHandles[::-1], loc='upper right', frameon=True)
    
    plt.xlabel(f'% Strain replacements per {by.replace("Participant ID", "participant").lower()}')
    plt.ylabel(f'% {"Species" if by == "Participant ID" else "Participants"}')
    plt.xlim([0, 100])
    plt.ylim([0, 100])

    _, p = mannwhitneyu(x=df.loc[df[hue] == hue_order[0], '%'].tolist(),
                        y=df.loc[df[hue] == hue_order[1], '%'].tolist(),
                        use_continuity=True, alternative='two-sided', axis=0, method='auto')
    plt.text(x=0.865, y=0.8, s='p'+r'$\leq$'+f'{p:.0e}', transform=ax.transAxes)

    plt.text(x=0, y=1.03, s='a' if by == 'Participant ID' else 'b', transform=ax.transAxes, size=20, weight='bold')

    plt.savefig(f'Figure 6{"a" if by == "Participant ID" else "b"}')

In [ ]:
for by in ['Participant ID', 'Species']:
    
    df = get_data(by)
    
    plt.figure()
    ax = sns.boxplot(x='n_bin', y='%', hue=hue, hue_order=hue_order, data=df, palette=colors, boxprops={'alpha': 0.7}, fliersize=0)
    box_pairs = [((b, hue_order[0]), (b, hue_order[1])) for b in df['n_bin'].unique().dropna()]
    ax, test_results = add_stat_annotation(ax, x='n_bin', y='%', hue=hue, data=df, box_pairs=box_pairs, test='Mann-Whitney', text_format='simple', comparisons_correction=None)
    sns.stripplot(x='n_bin', y='%', hue=hue, hue_order=hue_order, data=df, palette=colors, dodge=True, legend=False, color='lightgrey', s=8, alpha=0.3, ax=ax)

    corr = df.groupby(hue).apply(lambda g: pearsonr(g['n'], g['%']))
    l = ax.legend(title=False, loc='upper right', frameon=True)
    for handle in l.legendHandles:
        h = handle.get_label()
        r, p = corr.loc[h]
        r, p
        handle.set_label(f'{h}\nr={r:.2f}, ' + (f'p<1e{math.floor(math.log10(p))+1}' if p < 0.01 else f'p={p:.2f}'))
    plt.legend(handles=l.legendHandles, title=False, loc='upper right', frameon=True)

    new_labels = [t.get_text().replace('(', '').replace(']', '').replace(', ', '-') for t in ax.get_xticklabels()]
    ax.set_xticklabels(new_labels)
    
    plt.xlabel(f'Number of {"species" if by == "Participant ID" else "participants"} available for comparison\n{"Environments richness" if by == "Participant ID" else "Species prevalence"}')
    plt.ylabel(f'% Strain replacements per {by.replace("Participant ID", "participant").lower()}')
    
    plt.text(x=0, y=1.03, s='c' if by == 'Participant ID' else 'd', transform=ax.transAxes, size=20, weight='bold')
    
    plt.savefig(f'Figure 6{"c" if by == "Participant ID" else "d"}')

**Read the discussion section of the paper**